In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Read fingerprint file
df = pd.read_csv("feature.csv")

X=df.drop(['compound_ID','active'],axis=1)


y = df['active']

print(X)
print(y)



x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed)


import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV 
import pandas as pd
import numpy as np
import math


model = xgb.XGBRegressor()

param_dict = {
         'n_estimators':range(1,301,1),
         'max_depth': range(1,50,1), 
         'learning_rate':[0.00001, 0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1],
         'min_child_weight': [1, 2, 3, 4, 5, 6, 7],
         'gamma':[0.0001, 0.001, 0.01, 0.1, 1]
         
         }
              
grid =  RandomizedSearchCV(model, param_dict, cv = 10, verbose=1 , scoring='r2',random_state = seed)

grid.fit(x_train,y_train)

best_estimator = grid.best_estimator_
best_score =grid.best_score_ 
cv_result = grid.cv_results_
print(best_estimator)
print(best_score)
print(cv_result)

Q2CV = best_score

scoreR2tra = best_estimator.score(x_train,y_train)
scoreR2test = best_estimator.score(x_test, y_test)
print(f"Q2CV:{Q2CV}\nR2tra:{scoreR2tra}\nR2test:{scoreR2test}")

train_pre = best_estimator.predict(x_train)
test_pre = best_estimator.predict(x_test)

MSEtra = mean_squared_error(train_pre, y_train)
RMSEtra = math.sqrt(MSEtra)
print("RMSEtra:",RMSEtra)
MSEtest = mean_squared_error(test_pre, y_test)
RMSEtest = math.sqrt(MSEtest)
print("RMSEtest:",RMSEtest)
y_pred= best_estimator.predict(x_test)
score = best_estimator.score(x_test, y_test)

data = {"index":["Q2CV","R2tra","R2test","RMSEtra","RMSEtest"],
        "value":[Q2CV,scoreR2tra,scoreR2test,RMSEtra,RMSEtest]
        
        }

result = pd.DataFrame(data)
result.to_excel("hERG_R-XGB-feature-result.xlsx")

import joblib
  
joblib.dump(best_estimator, 'hERG_R-XGB-feature.pkl')